# 04 — Content-Based Recommender (Experiments)

This notebook benchmarks several content-based recommenders for Steam games on a sampled user set. We use BM25-weighted metadata (tags, categories, developers), dense features (price, owners buckets), optional SVD, and multiple models:

- Popularity baseline.
- Content + popularity hybrids (alpha blend).
- Feature-kNN (precomputed neighbors on content features).
- LightFM hybrid (item features + implicit interactions) with a small sweep.
- Logistic regression scorer (pointwise, content+pop features with negative sampling).

Metrics: HitRate, Recall, NDCG at K (shared evaluator from `src.evaluation`). Seen items are excluded in recommendations.


## Setup
- Sample a subset of users for speed (adjust `SAMPLE_USERS`).
- Build BM25 + dense features; optional SVD.
- Run sweeps on alphas, kNN neighbors, LightFM (small), and logistic scorer.


In [1]:
import os

os.chdir('/home/alyx/Documents/RS/Project')

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Optional, Tuple

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from scipy.sparse import csr_matrix, hstack
from tqdm import tqdm

from src import config
from src.evaluation import build_ground_truth, evaluate_model
from src.models.popularity import PopularityRecommender
from src.models.content_based import ContentHybridRecommender

In [3]:
# Experiment params (tuned for sampled runs)
SAMPLE_USERS = 2000
MIN_INTERACTIONS = 10
ALPHAS = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
KNN_NEIGHBORS = [50, 100, 150]
VOCAB_TAGS = 4000
VOCAB_CATEGORIES = 2000
VOCAB_DEVELOPERS = 1000
USE_SVD = True
SVD_COMPONENTS = [128, 256]
BM25_K1 = 1.6
BM25_B = 0.75
LIGHTFM_FACTORS = [32, 64]
LIGHTFM_EPOCHS = [5]
LIGHTFM_LOSSES = ["warp"]
LOGREG_NEG_PER_POS = 2
LOGREG_MAX_USERS = 2000
RANDOM_STATE = 42


## Load processed data


In [4]:
USER_COL = config.USER_COL
ITEM_COL = config.ITEM_COL

train_df = pd.read_parquet(config.PROCESSED_DATA_DIR / "train_interactions.parquet")
test_df = pd.read_parquet(config.PROCESSED_DATA_DIR / "test_interactions.parquet")
item_features = pd.read_parquet(config.PROCESSED_DATA_DIR / "item_features.parquet").fillna(0)
games_meta = pd.read_parquet(config.PROCESSED_DATA_DIR / "games_metadata.parquet")

print("Raw shapes:", train_df.shape, test_df.shape, item_features.shape, games_meta.shape)

Raw shapes: (9117646, 2) (44021, 2) (89618, 4239) (89618, 47)


## Sample users (for speed)


In [5]:
user_counts = train_df[USER_COL].value_counts()
eligible_users = user_counts[user_counts >= MIN_INTERACTIONS].index

if SAMPLE_USERS:
    rng = np.random.default_rng(RANDOM_STATE)
    sample_size = min(SAMPLE_USERS, len(eligible_users))
    sampled_users = rng.choice(eligible_users, size=sample_size, replace=False)
    train_df = train_df[train_df[USER_COL].isin(sampled_users)].copy()
    test_df = test_df[test_df[USER_COL].isin(sampled_users)].copy()
    print(f"Sampled {sample_size} users -> train {train_df.shape}, test {test_df.shape}")
else:
    train_df = train_df[train_df[USER_COL].isin(eligible_users)].copy()
    test_df = test_df[test_df[USER_COL].isin(eligible_users)].copy()
    print(f"Using all eligible users -> train {train_df.shape}, test {test_df.shape}")

Sampled 2000 users -> train (454394, 2), test (2000, 2)


## Feature helpers (BM25 + dense + optional SVD)


In [6]:
def bm25_block(series: pd.Series, max_features: int, k1: float = BM25_K1, b: float = BM25_B):
    texts = series.fillna("").astype(str).tolist()
    vec = CountVectorizer(max_features=max_features)
    X = vec.fit_transform(texts)
    tf = X
    dl = np.asarray(tf.sum(axis=1)).ravel()
    avg_dl = dl.mean() + 1e-8
    idf = np.log((tf.shape[0] - tf.astype(bool).sum(axis=0) + 0.5) / (tf.astype(bool).sum(axis=0) + 0.5)) + 1
    idf = np.asarray(idf).ravel()
    denom = tf + k1 * (1 - b + b * (dl / avg_dl))[:, None]
    numer = tf.multiply(k1 + 1)
    bm25 = numer.multiply(1 / denom)
    bm25 = bm25.multiply(idf)
    return bm25.tocsr()

def build_feature_matrix(base_feats: pd.DataFrame, meta: pd.DataFrame, use_svd: bool, svd_components: Optional[int]):
    items = base_feats[ITEM_COL].astype(int).tolist()
    meta_aligned = meta.set_index(ITEM_COL).reindex(base_feats[ITEM_COL]).reset_index()

    blocks = []
    # Base dense
    blocks.append(csr_matrix(base_feats.drop(columns=[ITEM_COL]).to_numpy(dtype=np.float32)))

    # Price/owners buckets
    price_col = config.PRICE_COL
    if price_col in meta_aligned.columns:
        prices = pd.to_numeric(meta_aligned[price_col], errors="coerce").fillna(0)
        bins = [0, 1, 5, 10, 20, 50, 100, np.inf]
        labels = [f"price_bin_{i}" for i in range(len(bins)-1)]
        price_bins = pd.get_dummies(pd.cut(prices, bins=bins, labels=labels, include_lowest=True))
    else:
        price_bins = pd.DataFrame(index=meta_aligned.index)

    if "estimated_owners" in meta_aligned.columns:
        owners_raw = meta_aligned["estimated_owners"].fillna("")
        def parse_owner(val):
            if isinstance(val, str) and "-" in val:
                try:
                    low = val.split("-")[0].replace(",", "").strip()
                    return float(low)
                except Exception:
                    return np.nan
            try:
                return float(val)
            except Exception:
                return np.nan
        owners_num = owners_raw.apply(parse_owner)
        bins = [0, 1e3, 1e4, 1e5, 1e6, 1e7, np.inf]
        labels = [f"owners_bin_{i}" for i in range(len(bins)-1)]
        owner_bins = pd.get_dummies(pd.cut(owners_num, bins=bins, labels=labels, include_lowest=True))
    else:
        owner_bins = pd.DataFrame(index=meta_aligned.index)

    extra_dense = pd.concat([price_bins, owner_bins], axis=1).fillna(0)
    blocks.append(csr_matrix(extra_dense.to_numpy(dtype=np.float32)))

    # BM25 text blocks
    if "categories" in meta_aligned.columns:
        blocks.append(bm25_block(meta_aligned["categories"], max_features=VOCAB_CATEGORIES))
    if "developers" in meta_aligned.columns:
        blocks.append(bm25_block(meta_aligned["developers"], max_features=VOCAB_DEVELOPERS))
    if "tags" in meta_aligned.columns:
        blocks.append(bm25_block(meta_aligned["tags"], max_features=VOCAB_TAGS))

    matrix = hstack(blocks).tocsr()

    if use_svd and svd_components:
        svd = TruncatedSVD(n_components=svd_components, random_state=RANDOM_STATE)
        matrix = svd.fit_transform(matrix)
        matrix = normalize(matrix)
        matrix = csr_matrix(matrix)

    matrix = normalize(matrix, norm="l2", axis=1)
    item_to_idx = {iid: i for i, iid in enumerate(items)}
    return items, item_to_idx, matrix

## Prepare features


In [7]:
items_in_split = set(train_df[ITEM_COL]) | set(test_df[ITEM_COL])
base_feats = item_features[item_features[ITEM_COL].isin(items_in_split)].copy().reset_index(drop=True)
meta_filtered = games_meta[games_meta[ITEM_COL].isin(items_in_split)].copy().reset_index(drop=True)

## Evaluation setup


In [8]:
ground_truth = build_ground_truth(test_df, user_col=USER_COL, item_col=ITEM_COL)
users_eval = list(ground_truth.keys())
known_items_map = train_df.groupby(USER_COL)[ITEM_COL].apply(list).to_dict()
print(f"Users for eval: {len(users_eval)}")

Users for eval: 2000


## Feature-kNN helper (precomputed neighbors)


In [9]:
class PrecomputedFeatureKNN:
    def __init__(self, item_matrix: csr_matrix, item_ids: List[int], item_to_idx: Dict[int, int], max_neighbors: int = 200):
        self.item_matrix = item_matrix
        self.item_ids = item_ids
        self.item_to_idx = item_to_idx
        self.max_neighbors = min(max_neighbors, item_matrix.shape[0]-1)
        knn = NearestNeighbors(metric="cosine", n_neighbors=self.max_neighbors)
        knn.fit(item_matrix)
        distances, neighbors = knn.kneighbors(item_matrix, n_neighbors=self.max_neighbors)
        self.neighbors = neighbors
        self.sims = 1 - distances
        self.default_n_neighbors = self.max_neighbors

    def recommend(self, user_id: int, known_items: List[int], k: int) -> List[int]:
        if not known_items:
            return []
        known_idx = [self.item_to_idx[i] for i in known_items if i in self.item_to_idx]
        if not known_idx:
            return []
        scores = np.zeros(self.item_matrix.shape[0], dtype=np.float32)
        n_use = self.default_n_neighbors
        for idx in known_idx:
            neigh = self.neighbors[idx, :n_use]
            sim = self.sims[idx, :n_use]
            scores[neigh] += sim
        for idx in known_idx:
            scores[idx] = -np.inf
        top_idx = np.argpartition(scores, -k)[-k:]
        top_idx = top_idx[np.argsort(scores[top_idx])[::-1]]
        return [self.item_ids[i] for i in top_idx]

## Sweeps: hybrids, kNN, SVD


In [10]:
all_results = []
svd_grid = SVD_COMPONENTS if USE_SVD else [None]

for svd_comp in tqdm(svd_grid, desc="SVD settings"):
    print(f"=== Feature store (SVD={svd_comp}) ===")
    item_ids, item_to_idx, item_matrix = build_feature_matrix(
        base_feats=base_feats,
        meta=meta_filtered,
        use_svd=USE_SVD,
        svd_components=svd_comp,
    )
    pop_counts = train_df[ITEM_COL].value_counts()
    pop_ranking = pop_counts.index.tolist()
    pop_scores = np.zeros(len(item_ids), dtype=np.float32)
    max_pop = pop_counts.max()
    for iid, count in pop_counts.items():
        idx = item_to_idx.get(iid)
        if idx is not None:
            pop_scores[idx] = count / max_pop

    pop_model = PopularityRecommender(item_col=ITEM_COL)
    pop_model.fit(train_df)
    metrics_pop = evaluate_model(pop_model, ground_truth, users_eval, ks=[5, 10, 20], known_items=known_items_map)
    metrics_pop["model"] = "popularity"
    metrics_pop["svd"] = svd_comp
    all_results.append(metrics_pop)

    for alpha in tqdm(ALPHAS, desc=f"Alphas (SVD={svd_comp})"):
        model = ContentHybridRecommender(
            item_ids=item_ids,
            item_to_idx=item_to_idx,
            item_matrix=item_matrix,
            pop_scores=pop_scores,
            pop_ranking=pop_ranking,
            user_col=USER_COL,
            item_col=ITEM_COL,
            alpha=alpha,
        )
        model.fit(train_df)
        metrics = evaluate_model(model, ground_truth, users_eval, ks=[5, 10, 20], known_items=known_items_map)
        metrics["model"] = f"hybrid_alpha_{alpha}"
        metrics["svd"] = svd_comp
        all_results.append(metrics)

    max_k = max(KNN_NEIGHBORS)
    knn_cache = PrecomputedFeatureKNN(item_matrix=item_matrix, item_ids=item_ids, item_to_idx=item_to_idx, max_neighbors=max_k)
    for n_nb in tqdm(KNN_NEIGHBORS, desc=f"kNN (SVD={svd_comp})"):
        knn_cache.default_n_neighbors = min(n_nb, knn_cache.max_neighbors)
        metrics_knn = evaluate_model(knn_cache, ground_truth, users_eval, ks=[5, 10, 20], known_items=known_items_map)
        metrics_knn["model"] = f"feature_knn_{n_nb}"
        metrics_knn["svd"] = svd_comp
        all_results.append(metrics_knn)

all_results_df = pd.concat(all_results)
all_results_pivot = all_results_df.pivot_table(index=["model", "svd"], columns="k", values=["hit_rate", "recall", "ndcg"])
all_results_df.head(), all_results_pivot

SVD settings:   0%|                                                                          | 0/2 [00:00<?, ?it/s]

=== Feature store (SVD=128) ===



Alphas (SVD=128): 100%|████████████████████████████████████████████████████████████| 10/10 [01:15<00:00,  7.52s/it]

SVD settings:  50%|████████████████████████████████▌                                | 1/2 [02:58<02:58, 178.82s/it]

=== Feature store (SVD=256) ===



Alphas (SVD=256): 100%|████████████████████████████████████████████████████████████| 10/10 [02:07<00:00, 12.76s/it]

SVD settings: 100%|█████████████████████████████████████████████████████████████████| 2/2 [08:39<00:00, 259.60s/it]


(    k  hit_rate  recall      ndcg              model  svd
 0   5    0.0835  0.0835  0.058795         popularity  128
 1  10    0.1190  0.1190  0.070056         popularity  128
 2  20    0.1850  0.1850  0.086637         popularity  128
 0   5    0.0810  0.0810  0.058294  hybrid_alpha_0.05  128
 1  10    0.1230  0.1230  0.071708  hybrid_alpha_0.05  128,
                       hit_rate                      ndcg                      \
 k                           5       10      20        5         10        20   
 model             svd                                                          
 feature_knn_100   128   0.0215  0.0345  0.0600  0.014138  0.018388  0.024826   
                   256   0.0295  0.0495  0.0740  0.018658  0.025072  0.031180   
 feature_knn_150   128   0.0215  0.0355  0.0575  0.013438  0.018026  0.023563   
                   256   0.0300  0.0440  0.0710  0.019683  0.024143  0.030853   
 feature_knn_50    128   0.0230  0.0405  0.0625  0.015539  0.021267  0.026717 

## LightFM sweep (small)


In [11]:
try:
    from lightfm import LightFM
    from lightfm.data import Dataset as LFMDataset

    lfm_ds = LFMDataset()
    lfm_ds.fit(users=train_df[USER_COL].unique(), items=train_df[ITEM_COL].unique())
    interactions, _ = lfm_ds.build_interactions(train_df[[USER_COL, ITEM_COL]].itertuples(index=False, name=None))

    item_ids_lfm, item_to_idx_lfm, item_matrix_lfm = build_feature_matrix(
        base_feats=base_feats,
        meta=meta_filtered,
        use_svd=USE_SVD,
        svd_components=SVD_COMPONENTS[0] if USE_SVD else None,
    )
    lfm_item_features = csr_matrix(item_matrix_lfm)

    user_id_map, user_feature_map, item_id_map, _ = lfm_ds.mapping()
    inv_item_map = {v: k for k, v in item_id_map.items()}

    for loss in LIGHTFM_LOSSES:
        for factors in LIGHTFM_FACTORS:
            for epochs in LIGHTFM_EPOCHS:
                print(f"LightFM loss={loss}, factors={factors}, epochs={epochs}")
                model_lfm = LightFM(loss=loss, no_components=factors, random_state=RANDOM_STATE)
                model_lfm.fit(interactions, item_features=lfm_item_features, epochs=epochs, num_threads=4)

                class LightFMWrapper:
                    def recommend(self, user_id: int, known_items: List[int], k: int) -> List[int]:
                        if user_id not in user_id_map:
                            return []
                        uid = user_id_map[user_id]
                        scores = model_lfm.predict(uid, np.arange(len(inv_item_map)), item_features=lfm_item_features)
                        ranked = np.argsort(-scores)
                        recs = []
                        known_set = set(known_items)
                        for idx in ranked:
                            itm = inv_item_map[idx]
                            if itm in known_set:
                                continue
                            recs.append(itm)
                            if len(recs) >= k:
                                break
                        return recs

                lfm_wrapper = LightFMWrapper()
                metrics_lfm = evaluate_model(lfm_wrapper, ground_truth, users_eval, ks=[5, 10, 20], known_items=known_items_map)
                metrics_lfm["model"] = f"lightfm_{loss}_f{factors}_e{epochs}"
                metrics_lfm["svd"] = SVD_COMPONENTS[0] if USE_SVD else None
                all_results_df = pd.concat([all_results_df, metrics_lfm])
                all_results_pivot = all_results_df.pivot_table(index=["model", "svd"], columns="k", values=["hit_rate", "recall", "ndcg"])
                display(metrics_lfm)
except Exception as e:
    print("LightFM not available or failed:", e)

LightFM loss=warp, factors=32, epochs=5


,k,hit_rate,recall,ndcg,model,svd
0,5,0.0075,0.0075,0.004270,lightfm_warp_f32_e5,128
1,10,0.0140,0.0140,0.006338,lightfm_warp_f32_e5,128
2,20,0.0245,0.0245,0.008954,lightfm_warp_f32_e5,128


LightFM loss=warp, factors=64, epochs=5


,k,hit_rate,recall,ndcg,model,svd
0,5,0.0055,0.0055,0.003290,lightfm_warp_f64_e5,128
1,10,0.0115,0.0115,0.005138,lightfm_warp_f64_e5,128
2,20,0.0210,0.0210,0.007551,lightfm_warp_f64_e5,128


## Logistic regression scorer


In [12]:
item_ids_lr, item_to_idx_lr, item_matrix_lr = build_feature_matrix(
    base_feats=base_feats,
    meta=meta_filtered,
    use_svd=USE_SVD,
    svd_components=SVD_COMPONENTS[0] if USE_SVD else None,
)

pop_counts_lr = train_df[ITEM_COL].value_counts()
pop_scores_lr = np.zeros(len(item_ids_lr), dtype=np.float32)
max_pop_lr = pop_counts_lr.max()
for iid, count in pop_counts_lr.items():
    idx = item_to_idx_lr.get(iid)
    if idx is not None:
        pop_scores_lr[idx] = count / max_pop_lr

user_profiles = {}
for user, grp in train_df.groupby(USER_COL):
    idxs = [item_to_idx_lr[i] for i in grp[ITEM_COL] if i in item_to_idx_lr]
    if not idxs:
        continue
    profile = item_matrix_lr[idxs].mean(axis=0)
    arr = np.asarray(profile).ravel()
    norm = np.linalg.norm(arr)
    if norm > 0:
        arr = arr / norm
    user_profiles[user] = arr

X_feat = []
y = []
users_iter = list(user_profiles.keys())[:LOGREG_MAX_USERS]
all_items_arr = np.array(item_ids_lr)
rng = np.random.default_rng(RANDOM_STATE)

for user in tqdm(users_iter, desc="LogReg samples"):
    known_list = train_df[train_df[USER_COL] == user][ITEM_COL].tolist()
    known_set = set(known_list)
    profile = user_profiles[user]
    for pos in known_list:
        idx = item_to_idx_lr.get(pos)
        if idx is None:
            continue
        content_score = float(item_matrix_lr[idx].dot(profile))
        X_feat.append([content_score, pop_scores_lr[idx]])
        y.append(1)
        neg_candidates = np.setdiff1d(all_items_arr, np.array(list(known_set)), assume_unique=True)
        if len(neg_candidates) == 0:
            continue
        neg_sample = rng.choice(neg_candidates, size=min(LOGREG_NEG_PER_POS, len(neg_candidates)), replace=False)
        for neg in neg_sample:
            nidx = item_to_idx_lr.get(int(neg))
            if nidx is None:
                continue
            content_score_neg = float(item_matrix_lr[nidx].dot(profile))
            X_feat.append([content_score_neg, pop_scores_lr[nidx]])
            y.append(0)

if X_feat:
    X_feat = np.array(X_feat, dtype=np.float32)
    y_arr = np.array(y, dtype=np.int8)
    clf = LogisticRegression(max_iter=200, n_jobs=4)
    clf.fit(X_feat, y_arr)

    def logreg_recommend_topk(uid: int, k: int) -> List[int]:
        if uid not in user_profiles:
            return []
        profile = user_profiles[uid]
        batch_size = 50000
        scores = np.empty(len(item_ids_lr), dtype=np.float32)
        for start in range(0, len(item_ids_lr), batch_size):
            end = min(start + batch_size, len(item_ids_lr))
            sub_idx = np.arange(start, end)
            content_scores = np.asarray(item_matrix_lr[sub_idx].dot(profile)).ravel()
            pop_sub = pop_scores_lr[sub_idx]
            feats = np.stack([content_scores, pop_sub], axis=1)
            proba = clf.predict_proba(feats)[:, 1]
            scores[sub_idx] = proba
        for itm in known_items_map.get(uid, []):
            idx = item_to_idx_lr.get(itm)
            if idx is not None:
                scores[idx] = -np.inf
        top_idx = np.argpartition(scores, -k)[-k:]
        top_idx = top_idx[np.argsort(scores[top_idx])[::-1]]
        return [item_ids_lr[i] for i in top_idx]

    metrics_lr = evaluate_model(None, ground_truth, users_eval, ks=[5, 10, 20], known_items=known_items_map, recommend_fn=logreg_recommend_topk, exclude_known=False)
    metrics_lr["model"] = f"logreg_neg{LOGREG_NEG_PER_POS}_users{LOGREG_MAX_USERS}"
    all_results_df = pd.concat([all_results_df, metrics_lr])
    all_results_pivot = all_results_df.pivot_table(index=["model", "svd"], columns="k", values=["hit_rate", "recall", "ndcg"])
    display(metrics_lr)
else:
    print("LogReg skipped: no training samples")

LogReg samples:   0%|                                                                     | 0/2000 [00:00<?, ?it/s]/tmp/ipykernel_64035/2470827003.py:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  content_score = float(item_matrix_lr[idx].dot(profile))
/tmp/ipykernel_64035/2470827003.py:53: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  content_score_neg = float(item_matrix_lr[nidx].dot(profile))
LogReg samples: 100%|██████████████████████████████████████████████████████████| 2000/2000 [14:03<00:00,  2.37it/s]


,k,hit_rate,recall,ndcg,model
0,5,0.0805,0.0805,0.050116,logreg_neg2_users2000
1,10,0.1260,0.1260,0.064907,logreg_neg2_users2000
2,20,0.1895,0.1895,0.080908,logreg_neg2_users2000


## Top models summary


In [13]:
for k in [5, 10, 20]:
    metric = "ndcg"
    summary = (
        all_results_df[all_results_df["k"] == k]
        .sort_values(by=metric, ascending=False)
        .reset_index(drop=True)
    )
    print("Top 5 models by NDCG@10:")
    display(summary.head(5))

Top 5 models by NDCG@10:


,k,hit_rate,recall,ndcg,model,svd
0,5,0.0885,0.0885,0.062581,hybrid_alpha_0.3,256.0
1,5,0.0875,0.0875,0.062347,hybrid_alpha_0.35,256.0
2,5,0.0870,0.0870,0.062211,hybrid_alpha_0.3,128.0
3,5,0.0870,0.0870,0.062073,hybrid_alpha_0.45,128.0
4,5,0.0865,0.0865,0.061942,hybrid_alpha_0.35,128.0


Top 5 models by NDCG@10:


,k,hit_rate,recall,ndcg,model,svd
0,10,0.1370,0.1370,0.077934,hybrid_alpha_0.45,256.0
1,10,0.1355,0.1355,0.077879,hybrid_alpha_0.4,256.0
2,10,0.1345,0.1345,0.077498,hybrid_alpha_0.35,256.0
3,10,0.1335,0.1335,0.077190,hybrid_alpha_0.45,128.0
4,10,0.1340,0.1340,0.077124,hybrid_alpha_0.4,128.0


Top 5 models by NDCG@10:


,k,hit_rate,recall,ndcg,model,svd
0,20,0.1875,0.1875,0.091003,hybrid_alpha_0.4,256.0
1,20,0.1890,0.1890,0.090932,hybrid_alpha_0.3,128.0
2,20,0.1875,0.1875,0.090868,hybrid_alpha_0.45,128.0
3,20,0.1885,0.1885,0.090867,hybrid_alpha_0.35,128.0
4,20,0.1870,0.1870,0.090709,hybrid_alpha_0.35,256.0


## Notes
- Adjust `SAMPLE_USERS` and grids for speed/coverage. For full data, use the HPC script.
- BM25 + SVD hybrids are strong; kNN, LightFM, and LogReg provide complementary baselines.
- Metrics: HitRate/Recall/NDCG@K (seen items filtered). With one held-out item per user, HitRate and Recall will match.
